# Live Video Demo

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras 
import os

In [2]:
import tensorflow as tf
from keras.layers import TFSMLayer

# Path to the SavedModel directory
model_path = r"C:\Users\Hp\Downloads\saved_model_dir"

# Load the model as a TensorFlow SavedModel Layer
model = TFSMLayer(model_path, call_endpoint='serving_default')

# The model can now be used for inference as a layer in another model or standalone inference.


In [4]:
import cv2
import numpy as np
import tensorflow as tf
from keras.layers import TFSMLayer  # Import TFSMLayer

# Load the trained model using TFSMLayer
model_path = r"C:\Users\Hp\Downloads\saved_model_dir"
model = TFSMLayer(model_path, call_endpoint='serving_default')  # Adjust to match your model's endpoint

font_scale = 1.5
font = cv2.FONT_HERSHEY_PLAIN

rectangle_bgr = (255, 255, 255)
img = np.zeros((500, 500), dtype=np.uint8)
text = 'Some text in the box'
(text_width, text_height) = cv2.getTextSize(text, font, fontScale=font_scale, thickness=1)[0]
text_offset_x = 10
text_offset_y = img.shape[0] - 25
box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width + 2 + text_offset_y - text_height - 2, text_offset_y + text_height + 2))
cv2.rectangle(img, box_coords[0], box_coords[1], rectangle_bgr, cv2.FILLED)
cv2.putText(img, text, (text_offset_x, text_offset_y), font, fontScale=font_scale, color=(0, 0, 0), thickness=1)

cap = cv2.VideoCapture(1)
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError('Cannot open webcam')

faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    ret, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, 1.1, 4)
    for x, y, w, h in faces:
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = frame[y:y + h, x:x + w]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        if roi_color.size > 0:
            face_roi = roi_color
            final_image = cv2.resize(face_roi, (224, 224))
            final_image = np.expand_dims(final_image, axis=0)
            final_image = final_image / 255.0  # Normalization

            # Prediction using TFSMLayer model
            prediction = model(final_image, training=False)
            
            # Adjust key according to model output
            prediction_tensor = prediction['dense_2']  # Replace 'dense_2' with the correct output key

            if isinstance(prediction_tensor, tf.Tensor):
                prediction_tensor = prediction_tensor.numpy()

            status = ''
            if np.argmax(prediction_tensor) == 0:
                status = 'Angry'
            elif np.argmax(prediction_tensor) == 1:
                status = 'Disgust'
            elif np.argmax(prediction_tensor) == 2:
                status = 'Fear'
            elif np.argmax(prediction_tensor) == 3:
                status = 'Happy'
            elif np.argmax(prediction_tensor) == 4:
                status = 'Sad'
            elif np.argmax(prediction_tensor) == 5:
                status = 'Surprise'
            else:
                status = 'Neutral'

            x1, y1, w1, h1 = 0, 0, 175, 175
            cv2.rectangle(frame, (x1, y1), (x1 + w1, y1 + h1), (0, 0, 0), -1)
            cv2.putText(frame, status, (x1 + int(w1 / 10), y1 + int(h1 / 2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, status, (100, 150), font, 3, (0, 0, 255), 2, cv2.LINE_4)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255))

    cv2.imshow('Face Emotion Recognition', frame)

    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
